## MIE424 (2021 Winter) Lab 2

Yuehuan He and Jake Mosseri 

In this lab, you will be using linear classifier to build a spam classifier. 

Many email services today provide spam filters that are able to classify emails into spam and non-spam email with high accuracy. You will use linear classifier to build your own spam filter.

You will be training a classifier to classify whether a given email, $x$, is spam ($y = 1$) or non-spam ($y = 0$). In particular, you need to convert each email into a feature vector $x \in R^n$. The following parts of the lab will walk you through how such a feature vector can be constructed from an email.

Dataset used:

- spamTrain.mat - Spam training set
- spamTest.mat - Spam test set



The dataset has been processed using the following feature extraction. n = # words in vocabulary list. Specifically, the feature $x_i \in \{0, 1\}$ for an email corresponds to whether the $i$-th word in the dictionary occurs in the email. That is, $x_i = 1$ if the $i$-th word is in the email and $x_i$ = 0 if the $i$-th word is not present in the email.



Preprocessing process:

-  Lower-casing: The entire email is converted into lower case, so that captialization is ignored (e.g., IndIcaTE is treated the same as Indicate).
- Stripping HTML: All HTML tags are removed from the emails. Many emails often come with HTML formatting; we remove all the HTML tags, so that only the content remains.
- Normalizing URLs: All URLs are replaced with the text “httpaddr”.
- Normalizing Email Addresses: All email addresses are replaced with the text “emailaddr”.
- Normalizing Numbers: All numbers are replaced with the text “number”.
- Normalizing Dollars: All dollar signs are replaced with the text “dollar”.
- Word Stemming: Words are reduced to their stemmed form. For ex- ample, “discount”, “discounts”, “discounted” and “discounting” are all replaced with “discount”. Sometimes, the Stemmer actually strips off additional characters from the end, so “include”, “includes”, “included”, and “including” are all replaced with “includ”.
- Removal of non-words: Non-words and punctuation have been re- moved. All white spaces (tabs, newlines, spaces) have all been trimmed to a single space character.

After preprocessing the emails, we have a list of words for each email. The next step is to choose which words we would like to use in our classifier and which we would want to leave out.

For this example, we have chosen only the most frequently occuring words as our set of words considered (the vocabulary list). Since words that occur rarely in the training set are only in a few emails, they might cause the model to overfit our training set. The complete vocabulary list is in the file vocab.txt and also shown in the figure below. Our vocabulary list was selected by choosing all words which occur at least a 100 times in the spam corpus, resulting in a list of 1899 words. In practice, a vocabulary list with about 10,000 to 50,000 words is often used.


![Encoding process](img/encoding.png)


Given the vocabulary list, we can now map each word in the preprocessed emails into a list of word indices that contains the index of the word in the vocabulary list. The following figure shows the mapping for the sample email. Specifically, in the sample email, the word “anyone” was first normalized to “anyon” and then mapped onto the index 86 in the vocabulary list.




![Encoded datapoint](img/encodedx.png)

### Load Necessary Libraries

In [2]:
try:
    import nltk
except ImportError:
    !pip install nltk
    import nltk
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from scipy.io import loadmat

### Import Dataset

Now we load a preprocessed training dataset that will be used to train a linear classifier. spamTrain.mat contains training examples of spam and non-spam email, while spamTest.mat contains test examples. Each original email was processed and converted into a vector $x_{(i)} \in R^{1899}$.


In [2]:
# Load the Spam Email dataset
train_data = loadmat('spamTrain.mat')
test_data = loadmat('spamTest.mat')
X_train = train_data['X']
Y_train = train_data['y'].ravel()
X_test  = test_data['Xtest']
Y_test  = test_data['ytest'].ravel()

In [3]:
# Load vocabulary list:
vocabulary = []
with open('vocab.txt') as f:
    for line in f:
        idx, word = line.split('\t')
        vocabulary.append(word.strip())

Consider the following example email: 

*Anyone knows how much it costs to host a web portal ?*
*Well, it depends on how many visitors youre expecting. This can be anywhere from less than 10 bucks a month to a couple of 100. You should checkout http://www.rackspace.com/ or perhaps Amazon EC2 if youre running something big..*
*To unsubscribe yourself from this mailing list, send an email to:
groupname-unsubscribe@egroups.com*

The encoded representation of this email is given in the first figure:

[86, 916, 794, 1077, ...]

To check this:

In [4]:
vocabulary[85]

'anyon'

In [5]:
vocabulary[915]

'know'

Recap from Lab 1:

- How to find the size of training set and test set?

In [6]:
print('There are',X_train.shape[0],'training examples and',X_test.shape[0],'test examples')

There are 4000 training examples and 1000 test examples


### Construct and Train a Linear Classifier

After loading the dataset, we then proceed to train a linear classifier with hinge loss to classify between spam ($y = 1$) and non-spam ($y = 0$) emails.

Here we use the `SGDClassifier` in sklearn package to construct and train a linear classifier:

class sklearn.linear_model.SGDClassifier(loss='hinge', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

Parameters:

- **loss**: str, default=’hinge’
    - The loss function to be used. Defaults to ‘hinge’.
    - The possible options are ‘hinge’, ‘squared_hinge’, etc.
- **penalty**: {‘l2’, ‘l1’, ‘elasticnet’}, default=’l2’
    - The penalty (aka regularization term) to be used. Defaults to ‘l2’ which is the standard regularizer for linear SVM models. ‘l1’ and ‘elasticnet’ might bring sparsity to the model (feature selection) not achievable with ‘l2’.
- **alpha**: float, default=0.0001
    - Constant that multiplies the regularization term. The higher the value, the stronger the regularization. 


See more in its [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)



In [7]:
clf = make_pipeline(StandardScaler(),SGDClassifier(alpha=0,learning_rate='invscaling',eta0=0.01,verbose=1))

# Here alpha = 0 is used to not apply any penalty. Learning rate then cannot take the default value 'optimal'
# and an initial stepsize needs to be specified.

In [8]:
clf.fit(X_train, Y_train)

-- Epoch 1
Norm: 0.89, NNZs: 1895, Bias: -0.203625, T: 4000, Avg. loss: 0.155137
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 0.92, NNZs: 1895, Bias: -0.228109, T: 8000, Avg. loss: 0.069924
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 0.93, NNZs: 1895, Bias: -0.239498, T: 12000, Avg. loss: 0.053908
Total training time: 0.04 seconds.
-- Epoch 4
Norm: 0.95, NNZs: 1895, Bias: -0.247966, T: 16000, Avg. loss: 0.045720
Total training time: 0.05 seconds.
-- Epoch 5
Norm: 0.96, NNZs: 1895, Bias: -0.254312, T: 20000, Avg. loss: 0.040352
Total training time: 0.06 seconds.
-- Epoch 6
Norm: 0.97, NNZs: 1895, Bias: -0.258811, T: 24000, Avg. loss: 0.036693
Total training time: 0.07 seconds.
-- Epoch 7
Norm: 0.98, NNZs: 1895, Bias: -0.262675, T: 28000, Avg. loss: 0.033925
Total training time: 0.08 seconds.
-- Epoch 8
Norm: 0.98, NNZs: 1895, Bias: -0.265967, T: 32000, Avg. loss: 0.031756
Total training time: 0.09 seconds.
-- Epoch 9
Norm: 0.99, NNZs: 1895, Bias: -0.268194, T: 36000, Avg.

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier',
                 SGDClassifier(alpha=0, eta0=0.01, learning_rate='invscaling',
                               verbose=1))])

### Evaluate the fitted model

In [9]:
# Training accuracy
prediction_train = clf.predict(X_train)
print ('Train Accuracy:', np.mean(prediction_train == Y_train) * 100,'%')

Train Accuracy: 99.35000000000001 %


In [10]:
# Test accuracy
prediction_test = clf.predict(X_test)
print ('Test Accuracy:', np.mean(prediction_test == Y_test) * 100,'%')

Test Accuracy: 98.0 %


### Discussion

To better understand how the spam classifier works, we can inspect the parameters to see which words the classifier thinks are the most predictive of spam. The next step of this lab finds the parameters with the largest positive values in the classifier and displays the corresponding words. Thus, if an email contains words such as “click”, “remove”, “gurantee” etc., it is likely to be classified as spam.

In [11]:
coef = clf.named_steps['sgdclassifier'].coef_.ravel()
idx  = coef.argsort()[::-1].ravel()
print ('Top predictors of spam:')
for i in range(15):
    print (vocabulary[idx[i]], coef[idx[i]])

Top predictors of spam:
click 0.15901720423151255
remov 0.1388167927331402
our 0.12610588373517392
pleas 0.12048564555028425
guarante 0.10824668012348992
here 0.10682951252845208
basenumb 0.10501774129721947
opt 0.10378550130227342
nbsp 0.09059284919957059
you 0.0903798905989363
credit 0.08933330452503457
monthli 0.08498193817992589
dollar 0.08391510953575806
your 0.08017809026538159
paid 0.07902300282602404


Exercise:
- try train the model with other loss functions;
- find an email of your own and see if the fitted model can classify it correctly (the following helper functions might be useful in preprocessing your email content)
- train the model with a subset of the training data and see how the test accuracies compare, e.g. with 100% of training data vs with 50%.

In [14]:
import re
from nltk import PorterStemmer

def split(delimiters, string, maxsplit=0):
    pattern = '|'.join(map(re.escape, delimiters))
    return re.split(pattern, string, maxsplit)

def process_email(email_contents,vocabulary):
    """
    Preprocesses a the body of an email and returns a list of word indices.
    Parameters
    ----------
    email_contents : string
        The email content.
    Returns
    -------
    list
        A list of word indices.
    """
    email_contents = email_contents.lower()
    email_contents = re.sub('<[^<>]+>', ' ', email_contents)
    email_contents = re.sub('[0-9]+', 'number', email_contents)
    email_contents = re.sub('(http|https)://[^\s]*', 'httpaddr', email_contents)
    email_contents = re.sub('[^\s]+@[^\s]+', 'emailaddr', email_contents)
    email_contents = re.sub('[$]+', 'dollar', email_contents)

    words = split(""" @$/#.-:&*+=[]?!(){},'">_<;%\n\r""", email_contents)
    word_indices = []
    stemmer = PorterStemmer()
    for word in words:
        word = re.sub('[^a-zA-Z0-9]', '', word)
        if word == '':
            continue
        word = stemmer.stem(word)
        if word in vocabulary:
            idx = vocabulary.index(word)
            word_indices.append(idx)

    return word_indices

Acknowledgements:

Adopted from problem set and course materials in Stanford CS229 and [nex3z](https://github.com/nex3z) implementation in python.